In [203]:
from __future__ import print_function
import torch
import pickle
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [226]:
output_label = "median_house_value"

# Use pandas to read CSV data as it contains various object types
# Feel free to use another CSV reader tool
# But remember that LabTS tests take Pandas Dataframe as inputs
data = pd.read_csv("housing.csv")
data

categorical_columns = list(data.select_dtypes(include=['object']).columns)[0]

data[categorical_columns] = data[categorical_columns].astype('category')
data.dtypes

longitude              float64
latitude               float64
housing_median_age     float64
total_rooms            float64
total_bedrooms         float64
population             float64
households             float64
median_income          float64
ocean_proximity       category
median_house_value     float64
dtype: object

In [205]:
# Check for missing column elements
print(data.isna().values.any())
# Check how many missings
print(f"{data.isna().sum().sum()} missing elements.")
# Check frequency of missing per column to decide whether to drop or substitute the missings.
data.isna().sum()/(len(data))*100

True
168 missing elements.


longitude             0.000000
latitude              0.000000
housing_median_age    0.000000
total_rooms           0.000000
total_bedrooms        1.017442
population            0.000000
households            0.000000
median_income         0.000000
ocean_proximity       0.000000
median_house_value    0.000000
dtype: float64

In [206]:
data.isna().sum()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        168
population              0
households              0
median_income           0
ocean_proximity         0
median_house_value      0
dtype: int64

In [207]:
# We find the total_bedrooms is the only column with missings.
# We confirm this by retrieving columns with missing elements:
print(data.loc[:, data.isnull().any()].columns)
# Drop rows with missings.
print(data.dropna())
# 16512 - 16344 = 168

Index(['total_bedrooms'], dtype='object')
       longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
0        -117.61     34.13                21.0       8416.0          1386.0   
1        -117.37     33.98                52.0        201.0            44.0   
2        -118.34     33.89                36.0       2274.0           411.0   
3        -118.92     35.13                29.0       1297.0           262.0   
4        -121.80     37.23                18.0       3179.0           526.0   
...          ...       ...                 ...          ...             ...   
16507    -119.53     36.55                34.0       2065.0           343.0   
16508    -122.40     37.73                50.0       1947.0           411.0   
16509    -118.41     33.92                29.0       1436.0           401.0   
16510    -117.08     32.62                36.0       1674.0           309.0   
16511    -117.05     32.72                35.0       3669.0           617.0   

       po

In [208]:
# Rows that contain missing values for the total_bedrooms column:
missing_idx = data[data.isnull().any(axis=1)].index.tolist()
print(missing_idx)
data[data.isnull().any(axis=1)]

[43, 48, 109, 246, 364, 563, 674, 705, 794, 861, 864, 998, 1023, 1192, 1367, 1515, 1553, 1636, 1829, 1979, 1994, 2071, 2073, 2111, 2257, 2335, 2444, 2589, 2626, 2647, 2662, 2705, 3075, 3206, 3252, 3337, 3476, 3513, 3657, 3742, 3851, 3964, 3977, 4060, 4514, 4559, 4606, 4870, 4875, 4918, 4956, 4972, 5032, 5223, 5352, 5730, 5905, 6024, 6139, 6521, 6554, 6624, 6809, 6852, 6872, 7015, 7022, 7436, 7457, 7509, 7614, 7710, 7770, 7918, 8125, 8195, 8274, 8342, 8451, 8565, 8768, 8975, 9034, 9075, 9189, 9233, 9383, 9404, 9539, 9605, 9711, 9821, 9960, 10055, 10112, 10166, 10295, 10304, 10314, 10339, 10475, 10772, 10806, 10964, 11001, 11142, 11236, 11244, 11248, 11638, 11681, 11762, 11874, 11906, 11932, 12191, 12221, 12331, 12559, 12584, 12723, 12763, 12845, 12860, 13223, 13289, 13355, 13454, 13481, 13531, 13665, 13772, 13800, 13853, 13944, 13968, 14290, 14325, 14365, 14463, 14478, 14553, 14933, 14972, 15045, 15056, 15067, 15092, 15096, 15162, 15285, 15353, 15381, 15402, 15434, 15535, 15551, 15683, 

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity,median_house_value
43,-117.91,34.05,35.0,3189.0,NaN,1727.0,500.0,5.0758,<1H OCEAN,211100.0
48,-119.73,36.83,8.0,3602.0,NaN,1959.0,580.0,5.3478,INLAND,138800.0
109,-118.36,34.05,45.0,2283.0,NaN,1093.0,475.0,2.5658,<1H OCEAN,252000.0
246,-119.01,36.06,25.0,1505.0,NaN,1392.0,359.0,1.6812,INLAND,47700.0
364,-118.20,33.92,45.0,1283.0,NaN,1025.0,248.0,3.2798,<1H OCEAN,141200.0
...,...,...,...,...,...,...,...,...,...,...
16052,-122.41,38.16,37.0,1549.0,NaN,863.0,275.0,2.7457,NEAR BAY,254700.0
16132,-122.44,37.73,39.0,1912.0,NaN,970.0,406.0,4.7813,NEAR BAY,275500.0
16219,-122.50,37.75,45.0,1620.0,NaN,941.0,328.0,4.3859,NEAR OCEAN,270200.0
16295,-117.91,34.02,22.0,6269.0,NaN,5587.0,1251.0,3.8201,<1H OCEAN,136200.0


In [228]:
# Impute/Fill the missing values with the mean:
# Could also impute the value using median, KNN or MICE.
data = data.fillna(data.mean())
# Show that the previously NaN values have been replaced by the mean.
data.iloc[missing_idx, :]
data.dtypes

longitude              float64
latitude               float64
housing_median_age     float64
total_rooms            float64
total_bedrooms         float64
population             float64
households             float64
median_income          float64
ocean_proximity       category
median_house_value     float64
dtype: object

In [210]:
# Check type of ocean proximity column
print("ocean_proximity column type:", data.ocean_proximity.dtype)
# Integer encode the textual labels that are ocean_proximity:
label_encoder = preprocessing.LabelEncoder()
ocp_idx = data.columns.get_loc('ocean_proximity')
integer_encoded = label_encoder.fit_transform(data.loc[:, 'ocean_proximity'])
data.insert(loc=ocp_idx+1, column='ocean_proximity_cat', value=integer_encoded)
# However label encoding of the ocean proximities introduces relation/
# comparison between various ocean promiximities due to the number sequencing.
# The algorithm might misunderstand that the data has a hierachy/order.
data

ocean_proximity column type: object


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity,ocean_proximity_cat,median_house_value
0,-117.61,34.13,21.0,8416.0,1386.0,4308.0,1341.0,4.4611,INLAND,1,164600.0
1,-117.37,33.98,52.0,201.0,44.0,130.0,24.0,2.0250,INLAND,1,125000.0
2,-118.34,33.89,36.0,2274.0,411.0,1232.0,423.0,5.3730,<1H OCEAN,0,244500.0
3,-118.92,35.13,29.0,1297.0,262.0,909.0,253.0,1.9236,INLAND,1,106300.0
4,-121.80,37.23,18.0,3179.0,526.0,1663.0,507.0,5.9225,<1H OCEAN,0,265800.0
...,...,...,...,...,...,...,...,...,...,...,...
16507,-119.53,36.55,34.0,2065.0,343.0,1041.0,313.0,3.2917,INLAND,1,111500.0
16508,-122.40,37.73,50.0,1947.0,411.0,1170.0,384.0,3.4769,NEAR BAY,3,238700.0
16509,-118.41,33.92,29.0,1436.0,401.0,674.0,343.0,3.6389,<1H OCEAN,0,275000.0
16510,-117.08,32.62,36.0,1674.0,309.0,818.0,307.0,3.4773,NEAR OCEAN,4,150400.0


In [211]:
# Create instance of one-hot-encoder.
# When the handle_unknown parameter is set to ‘ignore’ and an unknown category
# is encountered during transform, the resulting one-hot encoded columns for
# this feature will be all zeros. 
one_hot_encoder = preprocessing.OneHotEncoder(handle_unknown='ignore')
# Pass label encoded values of ocean proximity to the encoder.
one_hot_encodings = one_hot_encoder.fit_transform(data[['ocean_proximity_cat']])
print(one_hot_encodings)
encoded_df = pd.DataFrame(one_hot_mapping.toarray())
dum_df = pd.get_dummies(data, columns=['ocean_proximity'], prefix='Type_is')
data = dum_df
data

  (0, 1)	1.0
  (1, 1)	1.0
  (2, 0)	1.0
  (3, 1)	1.0
  (4, 0)	1.0
  (5, 1)	1.0
  (6, 0)	1.0
  (7, 3)	1.0
  (8, 3)	1.0
  (9, 3)	1.0
  (10, 1)	1.0
  (11, 0)	1.0
  (12, 1)	1.0
  (13, 3)	1.0
  (14, 0)	1.0
  (15, 1)	1.0
  (16, 1)	1.0
  (17, 3)	1.0
  (18, 0)	1.0
  (19, 0)	1.0
  (20, 3)	1.0
  (21, 0)	1.0
  (22, 0)	1.0
  (23, 0)	1.0
  (24, 1)	1.0
  :	:
  (16487, 4)	1.0
  (16488, 4)	1.0
  (16489, 0)	1.0
  (16490, 1)	1.0
  (16491, 0)	1.0
  (16492, 0)	1.0
  (16493, 3)	1.0
  (16494, 0)	1.0
  (16495, 1)	1.0
  (16496, 0)	1.0
  (16497, 3)	1.0
  (16498, 0)	1.0
  (16499, 4)	1.0
  (16500, 3)	1.0
  (16501, 0)	1.0
  (16502, 1)	1.0
  (16503, 1)	1.0
  (16504, 1)	1.0
  (16505, 0)	1.0
  (16506, 0)	1.0
  (16507, 1)	1.0
  (16508, 3)	1.0
  (16509, 0)	1.0
  (16510, 4)	1.0
  (16511, 4)	1.0


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity_cat,median_house_value,Type_is_<1H OCEAN,Type_is_INLAND,Type_is_ISLAND,Type_is_NEAR BAY,Type_is_NEAR OCEAN
0,-117.61,34.13,21.0,8416.0,1386.0,4308.0,1341.0,4.4611,1,164600.0,0,1,0,0,0
1,-117.37,33.98,52.0,201.0,44.0,130.0,24.0,2.0250,1,125000.0,0,1,0,0,0
2,-118.34,33.89,36.0,2274.0,411.0,1232.0,423.0,5.3730,0,244500.0,1,0,0,0,0
3,-118.92,35.13,29.0,1297.0,262.0,909.0,253.0,1.9236,1,106300.0,0,1,0,0,0
4,-121.80,37.23,18.0,3179.0,526.0,1663.0,507.0,5.9225,0,265800.0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16507,-119.53,36.55,34.0,2065.0,343.0,1041.0,313.0,3.2917,1,111500.0,0,1,0,0,0
16508,-122.40,37.73,50.0,1947.0,411.0,1170.0,384.0,3.4769,3,238700.0,0,0,0,1,0
16509,-118.41,33.92,29.0,1436.0,401.0,674.0,343.0,3.6389,0,275000.0,1,0,0,0,0
16510,-117.08,32.62,36.0,1674.0,309.0,818.0,307.0,3.4773,4,150400.0,0,0,0,0,1


In [212]:
# Drop ocean_proximity_cat column
data = data.drop('ocean_proximity_cat', axis=1)

In [213]:
# Now we need to move median_house_value test label column to the last column
# of the dataframe.
data1 = data.pop('median_house_value')
data['median_house_value'] = data1
data

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,Type_is_<1H OCEAN,Type_is_INLAND,Type_is_ISLAND,Type_is_NEAR BAY,Type_is_NEAR OCEAN,median_house_value
0,-117.61,34.13,21.0,8416.0,1386.0,4308.0,1341.0,4.4611,0,1,0,0,0,164600.0
1,-117.37,33.98,52.0,201.0,44.0,130.0,24.0,2.0250,0,1,0,0,0,125000.0
2,-118.34,33.89,36.0,2274.0,411.0,1232.0,423.0,5.3730,1,0,0,0,0,244500.0
3,-118.92,35.13,29.0,1297.0,262.0,909.0,253.0,1.9236,0,1,0,0,0,106300.0
4,-121.80,37.23,18.0,3179.0,526.0,1663.0,507.0,5.9225,1,0,0,0,0,265800.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16507,-119.53,36.55,34.0,2065.0,343.0,1041.0,313.0,3.2917,0,1,0,0,0,111500.0
16508,-122.40,37.73,50.0,1947.0,411.0,1170.0,384.0,3.4769,0,0,0,1,0,238700.0
16509,-118.41,33.92,29.0,1436.0,401.0,674.0,343.0,3.6389,1,0,0,0,0,275000.0
16510,-117.08,32.62,36.0,1674.0,309.0,818.0,307.0,3.4773,0,0,0,0,1,150400.0


In [214]:
# Apply min-max normalisation
# Convert to numpy array
column_labels = list(data.columns)
print(column_labels)
data_np = data.values
min_max_scaler = preprocessing.MinMaxScaler()
data_np_scaled = min_max_scaler.fit_transform(data_np)
# Convert back to Pandas DataFrame
data = pd.DataFrame(data_np_scaled, index=data.index, columns=data.columns)
data

['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income', 'Type_is_<1H OCEAN', 'Type_is_INLAND', 'Type_is_ISLAND', 'Type_is_NEAR BAY', 'Type_is_NEAR OCEAN', 'median_house_value']


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,Type_is_<1H OCEAN,Type_is_INLAND,Type_is_ISLAND,Type_is_NEAR BAY,Type_is_NEAR OCEAN,median_house_value
0,0.671315,0.168969,0.392157,0.213999,0.214929,0.120659,0.220358,0.273182,0.0,1.0,0.0,0.0,0.0,0.308454
1,0.695219,0.153029,1.000000,0.005061,0.006673,0.003560,0.003782,0.105178,0.0,1.0,0.0,0.0,0.0,0.226805
2,0.598606,0.143464,0.686275,0.057785,0.063625,0.034446,0.069396,0.336071,1.0,0.0,0.0,0.0,0.0,0.473196
3,0.540837,0.275239,0.549020,0.032937,0.040503,0.025393,0.041441,0.098185,0.0,1.0,0.0,0.0,0.0,0.188249
4,0.253984,0.498406,0.333333,0.080803,0.081471,0.046526,0.083210,0.373967,1.0,0.0,0.0,0.0,0.0,0.517113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16507,0.480080,0.426142,0.647059,0.052470,0.053073,0.029093,0.051307,0.192535,0.0,1.0,0.0,0.0,0.0,0.198970
16508,0.194223,0.551541,0.960784,0.049468,0.063625,0.032708,0.062983,0.205308,0.0,0.0,0.0,1.0,0.0,0.461237
16509,0.591633,0.146652,0.549020,0.036472,0.062073,0.018807,0.056241,0.216480,1.0,0.0,0.0,0.0,0.0,0.536082
16510,0.724104,0.008502,0.686275,0.042525,0.047796,0.022843,0.050321,0.205335,0.0,0.0,0.0,0.0,1.0,0.279176


In [215]:
data.columns = column_labels
data

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,Type_is_<1H OCEAN,Type_is_INLAND,Type_is_ISLAND,Type_is_NEAR BAY,Type_is_NEAR OCEAN,median_house_value
0,0.671315,0.168969,0.392157,0.213999,0.214929,0.120659,0.220358,0.273182,0.0,1.0,0.0,0.0,0.0,0.308454
1,0.695219,0.153029,1.000000,0.005061,0.006673,0.003560,0.003782,0.105178,0.0,1.0,0.0,0.0,0.0,0.226805
2,0.598606,0.143464,0.686275,0.057785,0.063625,0.034446,0.069396,0.336071,1.0,0.0,0.0,0.0,0.0,0.473196
3,0.540837,0.275239,0.549020,0.032937,0.040503,0.025393,0.041441,0.098185,0.0,1.0,0.0,0.0,0.0,0.188249
4,0.253984,0.498406,0.333333,0.080803,0.081471,0.046526,0.083210,0.373967,1.0,0.0,0.0,0.0,0.0,0.517113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16507,0.480080,0.426142,0.647059,0.052470,0.053073,0.029093,0.051307,0.192535,0.0,1.0,0.0,0.0,0.0,0.198970
16508,0.194223,0.551541,0.960784,0.049468,0.063625,0.032708,0.062983,0.205308,0.0,0.0,0.0,1.0,0.0,0.461237
16509,0.591633,0.146652,0.549020,0.036472,0.062073,0.018807,0.056241,0.216480,1.0,0.0,0.0,0.0,0.0,0.536082
16510,0.724104,0.008502,0.686275,0.042525,0.047796,0.022843,0.050321,0.205335,0.0,0.0,0.0,0.0,1.0,0.279176


longitude             float64
latitude              float64
housing_median_age    float64
total_rooms           float64
total_bedrooms        float64
population            float64
households            float64
median_income         float64
Type_is_<1H OCEAN     float64
Type_is_INLAND        float64
Type_is_ISLAND        float64
Type_is_NEAR BAY      float64
Type_is_NEAR OCEAN    float64
median_house_value    float64
dtype: object

In [216]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

NameError: name 'X' is not defined